<a href="https://colab.research.google.com/github/DPaletti/mida_acv/blob/main/mida_acv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sustainable Mobility: classification of electric scooters rides

In [ ]:
!git clone